In this notebook we compute the statistics from the predictions. The reason for using R instead of python in this occasion is to able to use the BayesFactor package

In [1]:
require(BayesFactor)
require(boot)

Loading required package: BayesFactor

Loading required package: coda

Loading required package: Matrix

************
Welcome to BayesFactor 0.9.12-4.2. If you have questions, please contact Richard Morey (richarddmorey@gmail.com).

Type BFManual() to open the manual.
************

Loading required package: boot



In [2]:
r2_score<-function(y_pred, y_true){
    ss_res<- sum((y_pred-y_true)**2)
    
    mu_true = mean(y_true)
    
    ss_tot<-sum((y_true-mu_true)**2)
    
    r2<-1 - ss_res/ss_tot
    r2
}

mean_absolute_error<-function(y_pred, y_true){
    mean(abs(y_pred-y_true))
}

mean_square_error<-function(y_pred, y_true){
    mean((y_pred-y_true)**2)
}

In [3]:
results_dir<-"../results/prediction_sensitivity"

In [4]:
target_cases<-c("mavgccaf_chgPerYr", "yeo-johnson_Site_of_Fastest_IMT_Max_Progr")

In [5]:
RANDOM_STATE<-0

In [6]:
res_df <- data.frame(target=character(),
                     contrast=character(), 
                     r=numeric(),
                     r_l_95CI = numeric(),
                     r_u_95CI = numeric(),
                     #pval = numeric(),
                     #MSE=numeric(),
                     MAE=numeric(),
                     R2=numeric(), 
                     BF10 = numeric(),
                     BF01 = numeric(),
                     stringsAsFactors=FALSE) 


set.seed(RANDOM_STATE)
row_ix<-1


for(contrast_id in c("look_neg_look_neut", "reg_neg_look_neg")){
    for(targ_case in target_cases){

    res_case<-c(targ_case, contrast_id)

    y_results_df<-read.csv(file.path(results_dir, targ_case, contrast_id, "y_preds.csv"))

    y_pred<-y_results_df[,"y_pred"]
    y_true<-y_results_df[,"y_true"]

    r2<-r2_score(y_pred, y_true)
    mse<-mean_square_error(y_pred, y_true)
    mae<-mean_absolute_error(y_pred, y_true)

    corrs.res<-cor.test(y_pred, y_true, alternative="greater")
    r<-corrs.res$estimate
    b_object <- boot(y_results_df, 
               statistic = function(data, i){cor(data[i, "y_pred"], data[i, "y_true"], method='pearson')},
               R = 1000)
    r_boots<-boot.ci(b_object, type = "perc")

    pv<-corrs.res$p.value

    r_l_95CI <- r_boots$perc[4]
    r_u_95CI <- r_boots$perc[5]

    bf_10<-as.vector(correlationBF( y_true, y_pred, nullInterval = c(0,1))[1])
    bf_01<-1/bf_10

    res_case<-c(res_case, round(c(r, r_l_95CI, r_u_95CI,  mae, r2, bf_10, bf_01), 3))
    res_df[row_ix, ]<- res_case
    row_ix =row_ix + 1

    }
}

res_df

,target,contrast,r,r_l_95CI,r_u_95CI,MAE,R2,BF10,BF01
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,mavgccaf_chgPerYr,look_neg_look_neut,-0.295,-0.881,-0.267,0.017,-0.032,0.044,22.523
2,yeo-johnson_Site_of_Fastest_IMT_Max_Progr,look_neg_look_neut,-0.48,-0.785,-0.348,0.051,-0.045,0.029,35.022
3,mavgccaf_chgPerYr,reg_neg_look_neg,-0.799,-0.938,-0.653,0.017,-0.016,0.017,57.171
4,yeo-johnson_Site_of_Fastest_IMT_Max_Progr,reg_neg_look_neg,-0.348,-0.518,-0.285,0.051,-0.044,0.038,26.072


In [8]:
#read.csv(file.path(results_dir, targ_case, contrast_id, "y_preds.csv"))